In [1]:
from dualcluster_memory.memory_system import DualClusterMemorySystem
from dotenv import load_dotenv
import os
import time

load_dotenv()

Deepseek_API_Key = os.getenv('SiliconFlow_API_KEY')

print("\n1. 初始化双簇记忆系统...")

memory_system = DualClusterMemorySystem(
    model_name="model/all-MiniLM-L6-v2",
    llm_backend="deepseek",
    llm_model="deepseek-ai/DeepSeek-V3",
    evo_threshold=3,  # 降低阈值，方便测试簇整合
    similarity_threshold=0.5,
    api_key=Deepseek_API_Key
)        
print("✅ 系统初始化成功")

d:\software\envs\intern\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-09 13:17:48,300 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-11-09 13:17:48,365 - chromadb.telemetry.product.posthog - ERROR - Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
2025-11-09 13:17:48,370 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: model/all-MiniLM-L6-v2



1. 初始化双簇记忆系统...


2025-11-09 13:18:09,592 - chromadb.telemetry.product.posthog - ERROR - Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
2025-11-09 13:18:09,613 - chromadb.telemetry.product.posthog - ERROR - Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
2025-11-09 13:18:09,613 - chromadb.telemetry.product.posthog - ERROR - Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
2025-11-09 13:18:09,633 - chromadb.telemetry.product.posthog - ERROR - Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
2025-11-09 13:18:09,633 - chromadb.telemetry.product.posthog - ERROR - Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
2025-11-09 13:18:09,651 - DualClusterMemorySystem - INFO - ChromaDB旧集合重置完成
2025-11-09 13:18:

✅ 系统初始化成功


In [2]:
test_content_1 = """
    问题：某快递公司需要为10个客户配送包裹，从仓库出发，每个客户仅访问一次，要求总行驶距离最短。
    建模逻辑：采用整数规划模型，定义二进制变量x_ij表示是否从节点i到节点j行驶，目标函数最小化总距离，约束包括：每个客户入度=1、出度=1，仓库出度=1、入度=1，消除子回路。
    代码实现：
    import gurobipy as gp
    from gurobipy import GRB

    def vrp_model(distance_matrix, num_customers):
        model = gp.Model("VRP")
        # 定义变量
        x = model.addVars(num_customers+1, num_customers+1, vtype=GRB.BINARY, name="x")
        # 目标函数
        model.setObjective(gp.quicksum(distance_matrix[i][j] * x[i][j] for i in range(num_customers+1) for j in range(num_customers+1) if i != j), GRB.MINIMIZE)
        # 约束：每个客户入度=1
        for j in range(1, num_customers+1):
            model.addConstr(gp.quicksum(x[i][j] for i in range(num_customers+1) if i != j) == 1)
        # 约束：每个客户出度=1
        for i in range(1, num_customers+1):
            model.addConstr(gp.quicksum(x[i][j] for j in range(num_customers+1) if i != j) == 1)
        # 仓库约束
        model.addConstr(gp.quicksum(x[0][j] for j in range(1, num_customers+1)) == 1)
        model.addConstr(gp.quicksum(x[i][0] for i in range(1, num_customers+1)) == 1)
        # 求解
        model.optimize()
        return model
    """

    # 测试用例2：带时间窗的VRP（与用例1同属建模簇，不同实现细节）
test_content_2 = """
    问题：某快递公司为10个客户配送，每个客户有时间窗[start_j, end_j]，需在时间窗内送达，总距离最短。
    建模逻辑：基于VRP整数规划模型，新增时间变量t_j表示到达客户j的时间，约束t_j >= t_i + travel_time_ij - M*(1-x_ij)，t_j >= start_j，t_j <= end_j。
    代码实现：
    import gurobipy as gp
    from gurobipy import GRB

    def vrp_time_window(distance_matrix, time_windows, num_customers):
        M = 1000  # 大M常数
        model = gp.Model("VRP_TimeWindow")
        x = model.addVars(num_customers+1, num_customers+1, vtype=GRB.BINARY, name="x")
        t = model.addVars(num_customers+1, vtype=GRB.CONTINUOUS, name="t")
        # 目标函数
        model.setObjective(gp.quicksum(distance_matrix[i][j] * x[i][j] for i, j in [(i,j) for i in range(num_customers+1) for j in range(num_customers+1) if i != j]), GRB.MINIMIZE)
        # 入度出度约束（同基础VRP）
        for j in range(1, num_customers+1):
            model.addConstr(gp.quicksum(x[i][j] for i in range(num_customers+1) if i != j) == 1)
        for i in range(1, num_customers+1):
            model.addConstr(gp.quicksum(x[i][j] for j in range(num_customers+1) if i != j) == 1)
        # 时间窗约束
        for j in range(num_customers+1):
            model.addConstr(t[j] >= time_windows[j][0])
            model.addConstr(t[j] <= time_windows[j][1])
        # 时间连续性约束
        for i in range(num_customers+1):
            for j in range(num_customers+1):
                if i != j:
                    model.addConstr(t[j] >= t[i] + distance_matrix[i][j] - M*(1 - x[i][j]))
        model.optimize()
        return model
    """

    # 测试用例3：背包问题（不同建模簇，对比测试）
test_content_3 = """
    问题：背包容量为50，有8个物品，每个物品有重量和价值，选择物品装入背包，使总价值最大且不超过容量。
    建模逻辑：0-1整数规划，变量x_i表示是否选择物品i，目标函数最大化总价值，约束总重量<=背包容量。
    代码实现：
    import gurobipy as gp
    from gurobipy import GRB

    def knapsack_model(weights, values, capacity):
        model = gp.Model("Knapsack")
        x = model.addVars(len(weights), vtype=GRB.BINARY, name="x")
        model.setObjective(gp.quicksum(values[i] * x[i] for i in range(len(weights))), GRB.MAXIMIZE)
        model.addConstr(gp.quicksum(weights[i] * x[i] for i in range(len(weights))) <= capacity)
        model.optimize()
        return model
    """

In [3]:
node_id_1 = memory_system.add_note(test_content_1)
#node_id_2 = memory_system.add_note(test_content_2)
#node_id_3 = memory_system.add_note(test_content_3)

2025-11-09 13:19:51,089 - httpx - INFO - HTTP Request: POST https://api.siliconflow.cn/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-09 13:19:51,092 - DualClusterMemorySystem - INFO - 建模逻辑：采用整数规划模型，定义二进制变量x_ij表示是否从节点i到节点j行驶，目标函数最小化总距离，约束包括：每个客户入度=1、出度=1，仓库出度=1、入度=1，消除子回路。
2025-11-09 13:19:51,092 - DualClusterMemorySystem - INFO - 完整代码：import gurobipy as gp
from gurobipy import GRB

def vrp_model(distance_matrix, num_customers):
    model = gp.Model("VRP")
    # 定义变量
    x = model.addVars(num_customers+1, num_customers+1, vtype=GRB.BINARY, name="x")
    # 目标函数
    model.setObjective(gp.quicksum(distance_matrix[i][j] * x[i][j] for i in range(num_customers+1) for j in range(num_customers+1) if i != j), GRB.MINIMIZE)
    # 约束：每个客户入度=1
    for j in range(1, num_customers+1):
        model.addConstr(gp.quicksum(x[i][j] for i in range(num_customers+1) if i != j) == 1)
    # 约束：每个客户出度=1
    for i in range(1, num_customers+1):
        model.addConstr(gp.quicksum(x[i][j] for j in range(num_custo

In [36]:
result = memory_system.full_retriever.search(query="背包容量为50，有8个物品，每个物品有重量和价值，选择物品装入背包，使总价值最大且不超过容量。", k=1)

Batches: 100%|██████████| 1/1 [00:00<00:00, 81.73it/s]
2025-11-06 18:21:57,904 - chromadb.telemetry.product.posthog - ERROR - Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


In [6]:
result = memory_system.model_retriever.search(query="背包容量为50，有8个物品，每个物品有重量和价值，选择物品装入背包，使总价值最大且不超过容量。", k=1)

Batches: 100%|██████████| 1/1 [00:00<00:00, 40.63it/s]


In [8]:
result

{'ids': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[]],
 'distances': [[]]}

In [7]:
1 / (1 + result["distances"][0][0])

IndexError: list index out of range

In [38]:
memory_system.implementation_retriever.search(query="背包容量为50，有8个物品，每个物品有重量和价值，选择物品装入背包，使总价值最大且不超过容量。", k=1)

Batches: 100%|██████████| 1/1 [00:00<00:00, 72.55it/s]


{'ids': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[]],
 'distances': [[]]}

In [16]:
node_1 = memory_system.read(node_id_1)
node_2 = memory_system.read(node_id_2)
node_3 = memory_system.read(node_id_3)
print(f"- 节点1（VRP）：建模簇ID={node_1.modeling_cluster_id}，实现簇ID={node_1.implementation_cluster_id}")
print(f"- 节点2（带时间窗VRP）：建模簇ID={node_2.modeling_cluster_id}，实现簇ID={node_2.implementation_cluster_id}")
print(f"- 节点3（背包问题）：建模簇ID={node_3.modeling_cluster_id}，实现簇ID={node_3.implementation_cluster_id}")

- 节点1（VRP）：建模簇ID=None，实现簇ID=None
- 节点2（带时间窗VRP）：建模簇ID=None，实现簇ID=None
- 节点3（背包问题）：建模簇ID=None，实现簇ID=None


In [13]:
node_2.id

'1305dc4d-2b0e-4ffb-88e1-e4bb058e75f0'